In [1]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

True

In [2]:
# CommaSeparatedListOutputParser를 초기화합니다.
# 이 파서는 LLM의 출력을 쉼표로 구분된 문자열 리스트로 변환합니다.
parser = CommaSeparatedListOutputParser()
print(parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [3]:
# 프롬프트 템플릿을 생성합니다.
# `{format_instructions}` 변수가 파서가 요구하는 출력 형식을 동적으로 삽입합니다.
prompt_template = """
당신은 뉴스 기사를 읽고 핵심 키워드를 나타내는 5개의 해시태그를 추천하는 전문가입니다.
해시태그는 쉼표로 구분된 목록으로 생성해야 합니다.

{format_instructions}

뉴스 기사 내용:
{news_article}
"""

# PromptTemplate 객체를 생성합니다.
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["news_article"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 사용자에게 받은 뉴스 기사 텍스트입니다.
example_news_article = """
SON 미스터리 '왜 동료들이 늘 따르나' LA 타임스 분석했는데 "손흥민, 16살 유럽 생활... 성격 좋고 영어 잘해"
손흥민(33)이 LA FC 동료들과 감독의 사랑을 듬뿍 받고 있다.
미국 LA타임스는 18일(한국시간) "손흥민은 16세부터 함부르크에서 뛰며 선수 생활을 시작했다. 때문에 미국에서도 쉽게 적응할 것이다"라며 "이미 EPL에서 이미 가치를 증명했을 뿐 아니라 영어가 유창하고 성격도 겸손하다. 동료들이 인정하고 따를 수밖에 없다"고 전했다.
손흥민은 지난 17일 미국 매사추세츠 폭스버러 질레트 스타디움에서 열린 뉴잉글랜드 레볼루션과의 '2025 미국 메이저리그사커(MLS)' 28라운드 원정에서 1도움을 올리며 2-0 승리를 이끌었다.
손흥민의 완벽한 전진 패스 도움으로 쐐기골을 넣은 마티유 초니에르는 "손흥민과 함께 뛸 수 있는 게 행복하다. 그와 함께 뛰면 경기가 더 쉬워진다"며 "그가 있어 훈련장에 오는 게 즐겁고 팀 분위기도 좋아졌다"고 말했다.
또 초니에르는 자신의 소셜미디어(SNS)에 손흥민을 태그하며 '우린 '티키타카'가 잘 맞아'라는 글을 올렸고, 여러 LA FC 동료들이 댓글을 달며 공감했다.
손흥민의 전방 압박 덕에 선제골을 넣은 마르코 델가도 인터뷰를 통해 "손흥민은 실력뿐 아니라 인성도 좋다. 동료들 모두 손흥민을 좋아한다"고 전했다.
스티브 체룬돌로 LA FC 감독도 손흥민의 합류로 팀 분위기도 한층 더 좋아졌다고 밝혔다. 뉴잉글랜드전 후 기자회견에서 그는 "손흥민은 팀에 늘 웃음을 준다. 의욕적이면서 동료들과 소통하는 선수가 존재하는 건 중요하다"며 "우리는 손흥민이 그동안 쌓아온 기술, 스피드, 피니시, 경험 등을 모두 얻었다. 그의 영입은 정말 놀라운 일이다"라고 전했다.
지난 7일 LA FC 유니폼을 입은 손흥민은 특유의 밝은 성격과 친화력으로 짧은 시간에 선수들과 친숙한 관계를 만들었다. LA FC가 공개한 첫 훈련 영상에서 동료들은 손흥민을 향해 물을 뿌리고 두들기는 장난을 치며 벌써 친해진 모습을 보여준다. 라커룸에서도 손흥민이 대화를 주도하며 분위기를 복돋으며 벌써 친숙한 리더십을 보여주고 있다.
손흥민의 합류 이후 LA FC는 성적도 좋다. 2경기 무패(1승1무) 행진을 달린다. 손흥민은 MLS 데뷔전이었던 지난 10일 시카고전에서 페널티킥(PK)을 얻어낸 데 이어 17일 뉴잉글랜드전에서 쐐기골을 도우며 MLS 데뷔 도움을 올렸다.
축구 통계 매체 '풋몹'에 따르면 이날 손흥민은 도움 외에도 슈팅 4회, 기회 창출 5회, 패스 성공률 88%(29/33회), 드리블 성공 4회 등 공격에서 맹위를 떨쳤다.
풋몹은 손흥민에게 경기 최고 평점인 8.5를 부여하며 경기 최우수 선수(MVP)로 선정했다. 양 팀 선수 통틀어 8점대 평점은 손흥민이 유일하다.
손흥민은 LA FC에서 계속 'SON 톱'으로 출격할 전망이다. 손흥민 포지션에 대한 질문을 받은 체룬돌로 감독은 "현재 팀 상황을 보면 손흥민은 측면보다 중앙에서 더 영향력을 발휘할 수 있다. 그가 오늘 중앙에서 어떤 역할을 할 수 있는지 잘 증명했다"고 전했다.
이어 "손흥민이 빠른 속도로 공격할 때, 잠시 플레이를 늦추며 완급 조절을 할 때, 안쪽에서 들어오는 윙어와 호흡을 맞출 때 등등 정말 인상적이었다"며 "측면 공격수 드니 부앙가와 스위칭하며 경기를 풀어갔다"고 덧붙였다.
"""

In [4]:
# 모델을 초기화합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# 프롬프트, 모델, 파서를 체인으로 연결합니다.
chain = prompt | model | parser

# 체인을 실행하고 해시태그 목록을 얻습니다.
hashtag_list = chain.invoke({"news_article": example_news_article})

In [ ]:
# 결과 출력
print("--- 추천 해시태그 ---")
for hashtag in hashtag_list:
    print(f"#{hashtag.strip()}")
print("\n" + "=" * 50 + "\n")
print("--- 생성된 객체 정보 ---")
print(f"타입: {type(hashtag_list)}")
print(f"내용: {hashtag_list}")

--- 추천 해시태그 ---
#손흥민
#LA FC
#팀 분위기
#MLS
#축구


--- 생성된 객체 정보 ---
타입: <class 'list'>
내용: ['손흥민', 'LA FC', '팀 분위기', 'MLS', '축구']
